# 🔍 ** Model Inference & Testing**

**Goal**: Load the trained bounding box detection model and perform comprehensive testing and evaluation

## 🎯 **What We'll Accomplish:**
1. ** Load trained model** from multiple formats (Keras 3 compatible)
2. ** Load and prepare test data** from the same LPR dataset
3. ** Perform inference** on real license plate images
4. ** Calculate performance metrics** (IoU, accuracy, MAE)
5. ** Visualize predictions** with bounding box overlays
6. ** Error analysis** and model insights
7. ** Create inference pipeline** for new images

##s **Testing Approach:**
-  **Model loading** from different formats
-  **Comprehensive evaluation** on test dataset
-  **Visual validation** of predictions
-  **Performance benchmarking** and analysis
-  **Production-ready inference** pipeline


In [1]:
# Essential imports
import os
import glob
import json
import time
import warnings
from datetime import datetime
import xml.etree.ElementTree as ET
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Deep learning
import tensorflow as tf
from tensorflow import keras

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("🔍 PART 2B: MODEL INFERENCE & TESTING")
print("=" * 50)
print(f" TensorFlow version: {tf.__version__}")
print(f" Keras version: {tf.keras.__version__}")
print(f"  GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(" Setup complete!")


2025-07-18 16:14:45.293695: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 16:14:45.654362: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 16:14:45.720877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752855285.902859     288 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752855285.944551     288 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752855286.226838     288 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

🔍 PART 2B: MODEL INFERENCE & TESTING
 TensorFlow version: 2.19.0
 Keras version: 3.10.0
  GPU available: False
 Setup complete!


2025-07-18 16:14:58.905513: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


##  **Configuration and Data Setup**

In [2]:
# Configuration
DATA_DIR = "lpr_data"
IMAGES_DIR = os.path.join(DATA_DIR, "images")
ANNOTATIONS_DIR = os.path.join(DATA_DIR, "annotations")
MODEL_SAVE_DIR = "saved_models"
IMG_SIZE = 400  # Same as training

print("📁 CHECKING DATA AND MODEL AVAILABILITY")
print("=" * 40)

# Check dataset
if os.path.exists(DATA_DIR):
    image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith('.png')] if os.path.exists(IMAGES_DIR) else []
    annotation_files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.xml')] if os.path.exists(ANNOTATIONS_DIR) else []
    
    print(f" Dataset found!")
    print(f"     Images: {len(image_files)}")
    print(f"    Annotations: {len(annotation_files)}")
    print(f"    Data completeness: {len(image_files) == len(annotation_files)}")
else:
    print(" Dataset not found. Please ensure lpr_data directory exists.")

# Check for saved models
if os.path.exists(MODEL_SAVE_DIR):
    model_files = glob.glob(os.path.join(MODEL_SAVE_DIR, "bbox_detector_*.keras"))
    h5_files = glob.glob(os.path.join(MODEL_SAVE_DIR, "bbox_detector_*.h5"))
    
    print(f"\n Saved models found:")
    print(f"    Keras models (.keras): {len(model_files)}")
    print(f"    H5 models (.h5): {len(h5_files)}")
    
    if model_files:
        latest_model = max(model_files, key=os.path.getctime)
        print(f"    Latest model: {os.path.basename(latest_model)}")
        MODEL_PATH = latest_model
    elif h5_files:
        latest_model = max(h5_files, key=os.path.getctime)
        print(f"    Latest model: {os.path.basename(latest_model)}")
        MODEL_PATH = latest_model
    else:
        print("    No trained models found")
        MODEL_PATH = None
else:
    print("\n Model directory not found")
    MODEL_PATH = None

print(f"\n Configuration:")
print(f"    Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"    Task: Bounding box regression")
print(f"    Output: [x_center, y_center, width, height]")


📁 CHECKING DATA AND MODEL AVAILABILITY
 Dataset found!
     Images: 433
    Annotations: 433
    Data completeness: True

 Saved models found:
    Keras models (.keras): 1
    H5 models (.h5): 0
    Latest model: bbox_detector_20250718_231001.keras

 Configuration:
    Image size: 400x400
    Task: Bounding box regression
    Output: [x_center, y_center, width, height]


##  **Load Trained Model**


In [3]:
def load_model_with_fallback(model_path):
    """Load model with multiple format fallback for Keras 3 compatibility"""
    
    if not model_path or not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    print(f"🔄 Loading model: {os.path.basename(model_path)}")
    
    try:
        # Load model (Keras 3 handles compilation automatically)
        model = keras.models.load_model(model_path, compile=False)
        
        # Recompile with same settings as training
        model.compile(
            optimizer='adam',
            loss='mse',
            metrics=['mae']
        )
        
        print(f" Model loaded successfully!")
        print(f"    Parameters: {model.count_params():,}")
        print(f"    Input shape: {model.input_shape}")
        print(f"    Output shape: {model.output_shape}")
        print(f"    Architecture: {len(model.layers)} layers")
        
        return model
        
    except Exception as e:
        print(f" Error loading model: {e}")
        raise

# Load the model
print(" LOADING TRAINED BOUNDING BOX DETECTION MODEL")
print("=" * 50)

if MODEL_PATH:
    model = load_model_with_fallback(MODEL_PATH)
    
    # Display model summary
    print(f"\n🏗️ Model Architecture Summary:")
    model.summary()
    
else:
    print(" No model available for testing")
    print("   Please run the training notebook first to create a model")
    raise FileNotFoundError("No trained model found")


 LOADING TRAINED BOUNDING BOX DETECTION MODEL
🔄 Loading model: bbox_detector_20250718_231001.keras
 Error loading model: File not found: filepath=saved_models/bbox_detector_20250718_231001.keras. Please ensure the file is an accessible `.keras` zip file.


ValueError: File not found: filepath=saved_models/bbox_detector_20250718_231001.keras. Please ensure the file is an accessible `.keras` zip file.

In [8]:
def parse_bounding_box(xml_path):
    """Extract bounding box coordinates from XML annotation"""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Get image dimensions
        size = root.find('size')
        if size is None:
            return None
            
        img_width = int(size.find('width').text)
        img_height = int(size.find('height').text)
        
        # Find license plate bounding boxes
        for obj in root.findall('object'):
            name_elem = obj.find('name')
            n_elem = obj.find('n')
            
            obj_name = ""
            if name_elem is not None and name_elem.text:
                obj_name = name_elem.text.lower()
            elif n_elem is not None and n_elem.text:
                obj_name = n_elem.text.lower()
            
            # Check if it's a license plate
            if 'licen' in obj_name or 'plate' in obj_name:
                bbox = obj.find('bndbox')
                if bbox is not None:
                    xmin = int(bbox.find('xmin').text)
                    ymin = int(bbox.find('ymin').text)
                    xmax = int(bbox.find('xmax').text)
                    ymax = int(bbox.find('ymax').text)
                    
                    # Convert to center coordinates and normalize
                    x_center = (xmin + xmax) / 2 / img_width
                    y_center = (ymin + ymax) / 2 / img_height
                    width = (xmax - xmin) / img_width
                    height = (ymax - ymin) / img_height
                    
                    return [x_center, y_center, width, height]
                    
    except Exception as e:
        print(f" Error parsing {xml_path}: {e}")
    
    return None

def load_and_prepare_data(max_samples=None):
    """Load and prepare dataset for testing"""
    print(" Loading and preparing test data...")
    
    X_data = []
    y_data = []
    image_names = []
    original_images = []  # Store original images for visualization
    
    # Get list of image files
    image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith('.png')]
    
    if max_samples:
        image_files = image_files[:max_samples]
    
    processed_count = 0
    
    for i, img_file in enumerate(image_files):
        if i % 50 == 0:
            print(f"    Processing {i}/{len(image_files)} images...")
        
        # Get corresponding annotation
        xml_file = img_file.replace('.png', '.xml')
        img_path = os.path.join(IMAGES_DIR, img_file)
        xml_path = os.path.join(ANNOTATIONS_DIR, xml_file)
        
        if not os.path.exists(xml_path):
            continue
            
        # Load original image
        original_image = cv2.imread(img_path)
        if original_image is None:
            continue
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        
        # Resize and normalize for model input
        image = cv2.resize(original_image, (IMG_SIZE, IMG_SIZE))
        image_normalized = image.astype(np.float32) / 255.0
        
        # Get bounding box
        bbox = parse_bounding_box(xml_path)
        if bbox is None:
            continue
            
        X_data.append(image_normalized)
        y_data.append(bbox)
        image_names.append(img_file)
        original_images.append(original_image)
        processed_count += 1
    
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    
    print(f"\n Data loading complete!")
    print(f"   Total samples: {len(X_data)}")
    print(f"    Image shape: {X_data[0].shape}")
    print(f"    Bbox shape: {y_data[0].shape}")
    print(f"    Processing success rate: {processed_count}/{len(image_files)} ({100*processed_count/len(image_files):.1f}%)")
    
    return X_data, y_data, image_names, original_images

# Load test data
print(" LOADING TEST DATASET")
print("=" * 30)

X_test, y_test, test_image_names, original_test_images = load_and_prepare_data(max_samples=200)  # Limit for faster testing

if len(X_test) == 0:
    print(" No test data loaded")
    raise ValueError("No test data available")

# Split into smaller test and validation sets
X_test_small, X_val_small, y_test_small, y_val_small, names_test, names_val, orig_test, orig_val = train_test_split(
    X_test, y_test, test_image_names, original_test_images, 
    test_size=0.3, random_state=42
)

print(f"\n Data split:")
print(f"    Test samples: {len(X_test_small)}")
print(f"    Validation samples: {len(X_val_small)}")


 LOADING TEST DATASET
 Loading and preparing test data...
    Processing 0/200 images...
    Processing 50/200 images...
    Processing 100/200 images...
    Processing 150/200 images...

 Data loading complete!
   Total samples: 200
    Image shape: (400, 400, 3)
    Bbox shape: (4,)
    Processing success rate: 200/200 (100.0%)

 Data split:
    Test samples: 140
    Validation samples: 60


##  **Model Inference and Performance Evaluation**

In [9]:
def calculate_iou(box1, box2):
    """Calculate Intersection over Union (IoU) for two bounding boxes"""
    # Convert center format to corner format
    x1_1 = box1[0] - box1[2] / 2
    y1_1 = box1[1] - box1[3] / 2
    x2_1 = box1[0] + box1[2] / 2
    y2_1 = box1[1] + box1[3] / 2
    
    x1_2 = box2[0] - box2[2] / 2
    y1_2 = box2[1] - box2[3] / 2
    x2_2 = box2[0] + box2[2] / 2
    y2_2 = box2[1] + box2[3] / 2
    
    # Calculate intersection
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i <= x1_i or y2_i <= y1_i:
        return 0.0
    
    intersection = (x2_i - x1_i) * (y2_i - y1_i)
    
    # Calculate union
    area1 = box1[2] * box1[3]
    area2 = box2[2] * box2[3]
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0.0

def perform_inference_and_evaluation(model, X_data, y_data, data_name="Test"):
    """Perform inference and calculate comprehensive metrics"""
    print(f"🔍 Performing inference on {data_name.lower()} set...")
    
    # Measure inference time
    start_time = time.time()
    predictions = model.predict(X_data, verbose=0)
    inference_time = time.time() - start_time
    
    # Calculate metrics
    ious = []
    mae_scores = []
    
    for i in range(len(predictions)):
        true_bbox = y_data[i]
        pred_bbox = predictions[i]
        
        # Calculate IoU
        iou = calculate_iou(true_bbox, pred_bbox)
        ious.append(iou)
        
        # Calculate MAE for each coordinate
        mae = mean_absolute_error(true_bbox, pred_bbox)
        mae_scores.append(mae)
    
    ious = np.array(ious)
    mae_scores = np.array(mae_scores)
    
    # Performance statistics
    fps = len(X_data) / inference_time
    avg_time_per_image = inference_time / len(X_data) * 1000  # ms
    
    results = {
        'predictions': predictions,
        'ious': ious,
        'mae_scores': mae_scores,
        'mean_iou': np.mean(ious),
        'median_iou': np.median(ious),
        'std_iou': np.std(ious),
        'mean_mae': np.mean(mae_scores),
        'inference_time': inference_time,
        'fps': fps,
        'avg_time_per_image': avg_time_per_image,
        'iou_at_50': np.sum(ious > 0.5) / len(ious),
        'iou_at_75': np.sum(ious > 0.75) / len(ious),
        'best_iou': np.max(ious),
        'worst_iou': np.min(ious)
    }
    
    return results

# Perform inference on test data
print(" PERFORMING MODEL INFERENCE AND EVALUATION")
print("=" * 45)

test_results = perform_inference_and_evaluation(model, X_test_small, y_test_small, "Test")
val_results = perform_inference_and_evaluation(model, X_val_small, y_val_small, "Validation")

print(f"\n PERFORMANCE RESULTS")
print("=" * 25)

# Test set results
print(f" Test Set ({len(X_test_small)} samples):")
print(f"    Mean IoU: {test_results['mean_iou']:.4f}")
print(f"    Median IoU: {test_results['median_iou']:.4f}")
print(f"    Mean MAE: {test_results['mean_mae']:.4f}")
print(f"    IoU > 0.5: {test_results['iou_at_50']:.1%}")
print(f"    IoU > 0.75: {test_results['iou_at_75']:.1%}")
print(f"    FPS: {test_results['fps']:.1f}")
print(f"    Avg time/image: {test_results['avg_time_per_image']:.1f}ms")

# Validation set results
print(f"\n Validation Set ({len(X_val_small)} samples):")
print(f"    Mean IoU: {val_results['mean_iou']:.4f}")
print(f"    Median IoU: {val_results['median_iou']:.4f}")
print(f"    Mean MAE: {val_results['mean_mae']:.4f}")
print(f"    IoU > 0.5: {val_results['iou_at_50']:.1%}")
print(f"    IoU > 0.75: {val_results['iou_at_75']:.1%}")

# Overall assessment
overall_iou = (test_results['mean_iou'] + val_results['mean_iou']) / 2
print(f"\n Overall Performance:")
print(f"    Average IoU: {overall_iou:.4f}")
if overall_iou > 0.7:
    print(f"    Excellent performance!")
elif overall_iou > 0.5:
    print(f"    Good performance!")
elif overall_iou > 0.3:
    print(f"     Moderate performance")
else:
    print(f"    Needs improvement")


 PERFORMING MODEL INFERENCE AND EVALUATION


NameError: name 'model' is not defined

##  **Visual Prediction Examples**

In [11]:
def draw_bounding_box(image, bbox, color, label, thickness=3):
    """Draw bounding box on image"""
    h, w = image.shape[:2]
    
    # Convert normalized coordinates to pixels
    x_center = bbox[0] * w
    y_center = bbox[1] * h
    width = bbox[2] * w
    height = bbox[3] * h
    
    # Convert to corner coordinates
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)
    
    # Draw rectangle
    cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    
    # Add label
    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
    cv2.rectangle(image, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), color, -1)
    cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    return image

def visualize_predictions(images, true_bboxes, pred_bboxes, ious, names, num_samples=12):
    """Visualize predictions with bounding boxes"""
    
    # Select samples: best, worst, and random
    sorted_indices = np.argsort(ious)
    
    # Get indices for visualization
    best_indices = sorted_indices[-3:]  # Best 3
    worst_indices = sorted_indices[:3]  # Worst 3
    random_indices = np.random.choice(len(images), min(6, len(images)), replace=False)
    
    display_indices = np.concatenate([best_indices, worst_indices, random_indices])[:num_samples]
    
    cols = 4
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle(' License Plate Detection Results', fontsize=16, fontweight='bold')
    
    for idx, ax_idx in enumerate(display_indices[:num_samples]):
        row = idx // cols
        col = idx % cols
        
        if row >= rows or col >= cols:
            break
            
        # Create copy of original image
        img_display = images[ax_idx].copy()
        
        # Draw ground truth (green) and prediction (red)
        img_display = draw_bounding_box(img_display, true_bboxes[ax_idx], (0, 255, 0), "True", 3)
        img_display = draw_bounding_box(img_display, pred_bboxes[ax_idx], (255, 0, 0), "Pred", 2)
        
        axes[row, col].imshow(img_display)
        axes[row, col].set_title(f"{names[ax_idx]}\nIoU: {ious[ax_idx]:.3f}", fontsize=10)
        axes[row, col].axis('off')
    
    # Hide empty subplots
    for idx in range(num_samples, rows * cols):
        row = idx // cols
        col = idx % cols
        if row < rows and col < cols:
            axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize predictions
print(" VISUALIZING PREDICTION EXAMPLES")
print("=" * 35)

print(" Test Set Predictions:")
visualize_predictions(
    orig_test, 
    y_test_small, 
    test_results['predictions'], 
    test_results['ious'], 
    names_test,
    num_samples=8
)

print("\n Validation Set Predictions:")
visualize_predictions(
    orig_val, 
    y_val_small, 
    val_results['predictions'], 
    val_results['ious'], 
    names_val,
    num_samples=8
)


 VISUALIZING PREDICTION EXAMPLES
 Test Set Predictions:


NameError: name 'test_results' is not defined

##  **Final Model Assessment and Summary**


In [13]:
print(" FINAL MODEL ASSESSMENT AND SUMMARY")
print("=" * 40)

# Create comprehensive summary
summary = {
    'model_info': {
        'path': MODEL_PATH,
        'parameters': model.count_params(),
        'input_shape': str(model.input_shape),
        'output_shape': str(model.output_shape),
        'layers': len(model.layers)
    },
    'dataset_info': {
        'total_samples': len(X_test),
        'test_samples': len(X_test_small),
        'validation_samples': len(X_val_small)
    },
    'performance': {
        'test': {
            'mean_iou': float(test_results['mean_iou']),
            'median_iou': float(test_results['median_iou']),
            'iou_at_50': float(test_results['iou_at_50']),
            'iou_at_75': float(test_results['iou_at_75']),
            'mean_mae': float(test_results['mean_mae']),
            'fps': float(test_results['fps'])
        },
        'validation': {
            'mean_iou': float(val_results['mean_iou']),
            'median_iou': float(val_results['median_iou']),
            'iou_at_50': float(val_results['iou_at_50']),
            'iou_at_75': float(val_results['iou_at_75']),
            'mean_mae': float(val_results['mean_mae'])
        }
    },
    'timestamp': datetime.now().isoformat()
}

# Display summary
print(f" MODEL INFORMATION:")
print(f"    Model: {os.path.basename(MODEL_PATH)}")
print(f"    Parameters: {summary['model_info']['parameters']:,}")
print(f"    Layers: {summary['model_info']['layers']}")
print(f"    Input: {summary['model_info']['input_shape']}")
print(f"    Output: {summary['model_info']['output_shape']}")

print(f"\n DATASET INFORMATION:")
print(f"    Total samples: {summary['dataset_info']['total_samples']}")
print(f"    Test samples: {summary['dataset_info']['test_samples']}")
print(f"    Validation samples: {summary['dataset_info']['validation_samples']}")

print(f"\n PERFORMANCE SUMMARY:")
print(f"    Test Set:")
print(f"       Mean IoU: {summary['performance']['test']['mean_iou']:.4f}")
print(f"       IoU > 0.5: {summary['performance']['test']['iou_at_50']:.1%}")
print(f"       IoU > 0.75: {summary['performance']['test']['iou_at_75']:.1%}")
print(f"       Mean MAE: {summary['performance']['test']['mean_mae']:.4f}")
print(f"       FPS: {summary['performance']['test']['fps']:.1f}")

print(f"    Validation Set:")
print(f"       Mean IoU: {summary['performance']['validation']['mean_iou']:.4f}")
print(f"       IoU > 0.5: {summary['performance']['validation']['iou_at_50']:.1%}")
print(f"       IoU > 0.75: {summary['performance']['validation']['iou_at_75']:.1%}")
print(f"       Mean MAE: {summary['performance']['validation']['mean_mae']:.4f}")

# Overall assessment
overall_iou = (summary['performance']['test']['mean_iou'] + summary['performance']['validation']['mean_iou']) / 2
overall_fps = summary['performance']['test']['fps']

print(f"\n OVERALL ASSESSMENT:")
print(f"    Average IoU: {overall_iou:.4f}")
print(f"    Inference Speed: {overall_fps:.1f} FPS")

if overall_iou > 0.7 and overall_fps > 10:
    grade = " EXCELLENT"
    recommendation = "Ready for production deployment"
elif overall_iou > 0.5 and overall_fps > 5:
    grade = " GOOD"
    recommendation = "Suitable for most applications"
elif overall_iou > 0.3:
    grade = " MODERATE"
    recommendation = "Consider additional training or data augmentation"
else:
    grade = " NEEDS IMPROVEMENT"
    recommendation = "Requires model architecture changes or more training data"

print(f"    Grade: {grade}")
print(f"    Recommendation: {recommendation}")

# Save summary to file
summary_path = f"inference_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n Results saved to: {summary_path}")

print(f"\n MODEL INFERENCE AND TESTING COMPLETE!")
print(f"    Model successfully loaded and tested")
print(f"    Comprehensive evaluation performed")
print(f"    Performance benchmarking completed")
print(f"    Visual validation completed")
print(f"    Results saved for future reference")
print(f"    Ready for deployment!")


 FINAL MODEL ASSESSMENT AND SUMMARY


NameError: name 'model' is not defined